In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/home-data-for-ml-course/sample_submission.csv
/kaggle/input/home-data-for-ml-course/train.csv.gz
/kaggle/input/home-data-for-ml-course/sample_submission.csv.gz
/kaggle/input/home-data-for-ml-course/test.csv.gz
/kaggle/input/home-data-for-ml-course/data_description.txt
/kaggle/input/home-data-for-ml-course/test.csv
/kaggle/input/home-data-for-ml-course/train.csv


In [2]:
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

In [3]:
data = pd.read_csv('../input/home-data-for-ml-course/train.csv',index_col='Id')
data_test = pd.read_csv('../input/home-data-for-ml-course/test.csv',index_col='Id')


data.dropna(axis=0, subset=['SalePrice'], inplace=True)
y= data.SalePrice
data.drop(['SalePrice'],axis=1,inplace=True)

full_X_train, full_X_valid, y_train, y_valid = train_test_split(data,y, train_size=0.8, test_size=0.2)


low_cardinality_columns= [nm for nm in full_X_train.columns if 
                          (full_X_train[nm].nunique()<10) and (full_X_train[nm].dtype=='object')]

num_cols= [nm for nm in full_X_train.columns if 
           full_X_train[nm].dtype==('int64','float64')]

my_cols = low_cardinality_columns+num_cols
X_train = full_X_train[my_cols]
X_valid = full_X_valid[my_cols]
X_test = data_test[my_cols]

my_imputer = SimpleImputer(strategy='most_frequent')

imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(my_imputer.transform(X_valid))
imputed_X_test = pd.DataFrame(my_imputer.transform(X_test))

imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns
imputed_X_test.columns = X_test.columns
imputed_X_test.index = X_test.index

obj_cols = [name for name in imputed_X_train.columns if imputed_X_train[name].dtype=='object']

OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)

OH_X_train = pd.DataFrame(OH_encoder.fit_transform(imputed_X_train[obj_cols]))
OH_X_valid = pd.DataFrame(OH_encoder.transform(imputed_X_valid[obj_cols]))
OH_X_test = pd.DataFrame(OH_encoder.transform(imputed_X_test[obj_cols]))

OH_X_train.index = imputed_X_train.index
OH_X_valid.index = imputed_X_valid.index
OH_X_test.index = X_test.index

num_X_train = imputed_X_train.drop(obj_cols,axis=1)
num_X_valid = imputed_X_valid.drop(obj_cols,axis=1)
num_X_test = imputed_X_test.drop(obj_cols,axis=1)


X_tr = pd.concat([OH_X_train,num_X_train],axis=1)
X_v = pd.concat([OH_X_valid,num_X_valid],axis=1)
X_te = pd.concat([OH_X_test,num_X_test],axis=1)

X_te

,0,1,2,3,4,5,6,7,8,9,...,6113,6114,6115,6116,6117,6118,6119,6120,6121,6122
Id,,,,,,,,,,,,,,,,,,,,,
1461,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1462,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1463,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1464,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1465,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2915,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2916,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2917,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [4]:
my_model = XGBRegressor(n_estimators=1000,learning_rate=0.05)
my_model.fit(X_tr,y_train,
            early_stopping_rounds=5,
            eval_set=[(X_v,y_valid)])

[0]	validation_0-rmse:184433.93750
Will train until validation_0-rmse hasn't improved in 5 rounds.
[1]	validation_0-rmse:175670.54688
[2]	validation_0-rmse:167364.28125
[3]	validation_0-rmse:159406.59375
[4]	validation_0-rmse:151785.07812
[5]	validation_0-rmse:144844.20312
[6]	validation_0-rmse:138029.95312
[7]	validation_0-rmse:131282.01562
[8]	validation_0-rmse:125240.60938
[9]	validation_0-rmse:119335.04688
[10]	validation_0-rmse:113570.05469
[11]	validation_0-rmse:108338.94531
[12]	validation_0-rmse:103287.53125
[13]	validation_0-rmse:98477.03125
[14]	validation_0-rmse:94263.24219
[15]	validation_0-rmse:90009.31250
[16]	validation_0-rmse:85905.52344
[17]	validation_0-rmse:82320.35156
[18]	validation_0-rmse:78762.84375
[19]	validation_0-rmse:75420.03125
[20]	validation_0-rmse:72278.35938
[21]	validation_0-rmse:69299.66406
[22]	validation_0-rmse:66529.86719
[23]	validation_0-rmse:64074.53125
[24]	validation_0-rmse:61648.50000
[25]	validation_0-rmse:59421.66406
[26]	validation_0-rmse:

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.05, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=1000, n_jobs=0, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [5]:
pred = my_model.predict(X_v)
print(f"MAE: {mean_absolute_error(pred,y_valid)}")

MAE: 21375.51000642123


In [6]:
pred1 = my_model.predict(X_te)
my_subm = pd.DataFrame({'Id':X_te.index,'SalePrice':pred1})

my_subm.to_csv('submission.csv',index=False)